In [9]:
from gensim.models import Word2Vec

from sklearn import cluster
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

from textblob.classifiers import NaiveBayesClassifier

import numpy as np
import numpy
import re
import nltk
from nltk.cluster import KMeansClusterer
from nltk.corpus import stopwords
stop_words = set(stopwords.words('french'))

In [10]:
positive = "accompagnement, apyretrique, apyrétique, RAS, a couché sa femme seul, visite de sa fille, mange en chambre avec sa fille, respiration plus régulière,\
pas de signe de detresse respiratoire, a bien dormi, les amies de mne X ont ramene des slips, belle-fille prévenue, calme ces 3 jours,\
absence d'épisode de selle liquide jusqu'alors ce jour, pas de douleur, se débrouille assez bien pour la toilette,\
Famille prévenue : Non, sa fille à apporté des affaires de toilette, a bien manger ce soir, toujours active,\
 refuse completement mon aide, dit pouvoir le faire seule, bien, RAS, Famille prévenue: nonMédecin prévenu: nonHospitalisation: non,\
demande de sa dame de compagnie, fille prévenue, moins douloureuse, pas d'essouflements constatés, se sentait bien, Hospitalisation: non,\
a tres bien dormie, semble moins crier, va mieux, discours cohérent, elle va mieux, visite de sa fille, allo sa soeur,\
ATB efficace car apyrethique, présente avec le groupe, a plaisir a les regarder travailler et a échanger, aide au dressage de la table,\
Satisfait, Etat : Actif, a passé une bonne nuit, Très gentille ce soir au coucher, commence à accepter qu'elle a besoin de notre aide,\
va globalement bien et n'est pas particulièrement dépressive, dort bien la nuit, fils averti, va bien pas de crise d angoiss,\
état psy chronique semble stable, semble s'être bien adaptée à la structure, plus de plaie, n a plus mal , Algoplus = 0/5,\
 a le sourire, beaucoup plus apaisée nous souris facilement, PLUS CALME, réussi a doucher, BU faite ce jour , est negative,\
Pas de nouvel incident ce jour.ai informé sa fille, non plaintif non douloureux, Bonne participation: Mme X a passé un tres bon moment ,\
 a sourit tout au long de la séance, discution, Ecouté rassuré, part chez son petit fils, très gentil et ça se passe très bien,\
 aime participé, nous la rassurons - ça devrait aller, vu sa famille, éveillée, agréable, discute volontiers, diminution de la posologie,\
 reactif conscience ok, ide prevenu, sa fracture c'était réduite. Arret de la mise en écharpe, se dit mieux, non douloureuse, etait souriante,\
Son fils lui a offert un poste lecteur CD, informé sa fille, agréable, discute, dort très bien, aucune conséquence, fille au tel,\
appel docteur, confort, a eu des selles ce jour, visite de son neveu, bonne nuit, rentrée à , mieux, s'apaise, enchantée, sourire, \
 fille a fait, rassurer, parler, mieux, gouter, carte vitale retrouvée, ses filles, A mieux dormit, mange bien, reconforté, rassuré,\
je viens de repasser la voir elle dort, il accepte, sa fille est informée, sortie, reunion équipe pluridisciplinaire,\
rdv cardiologue, toujours tres active, mieux, evoquer avec la fille, appel du Dr, sa fille nous a ramenée, accompagnée ce jour en salle de restaurant,\
a déjeuné en coimpagnie, fils appelé, pas de plainte, a tout mangé, a reçu ses bas de contention, pas de selle liquide,\
s'est progressivement détendue. à la fin de la séance, elle était bien et alerte"

nagative = "douleur,  Fatigué, et propos confus, marche difficilement, elle s endort avec ses vetements accepte finalement mon aide mais se rhabille ds la nuit,\
reste fatiguée et douloureuse malgré les ttt, elle a un bleu du coté gauche du hanche, Eresypel, Passive sur le temps des activités,\
 même si la conversation est difficile, n'a pas une très bonne image de sa personne, dyspnée encombrement trés endormi,\
a dormi tres tres profondement toute la matinee au moment du repas impossible de la faire lever, il faut le stimulé ++++ afin qu'il puisse participé a la toilette,\
algique 2 ou3 sur EN, a le moral dans les chaussettes, encore au plus mal avec crise de larmes et en me demandant de la tuer,\
ne pas trouver lunettes, retire son bracelet régulièrement,  a la voix prise, fatiguée, confusion persistante, agitation +++ difficultés d'élocution,\
trouvé madame X en larmes, mme très enervée et crie, trés désagréable, elle me crie dessus en me disant; que je suis une sauvage,\
se met à pleurer et commence à angoisser. Nous dis qu'elle n'arrivera pas à dormir sans son cachet, a fait une crise convulsive,\
etat douloureux, etat anxiaux, Asthénique, vois enrouée, GDS à 2/4Humeur triste. Souvent dans le noir dans sa chambre,\
nez coule toujours, resistant au moment de la douche, très endormi pendant les changes, pas encore de selles, Diarrhées persistantes malgré fin du traitement antibiotique,\
très réveillée et perturbée, à des hallucinations, se plaint de violente douleur a sa jambe droite ainsi qu'a son genou droit,\
SAD  arraché au pied de son lit, a aussi arraché sa protection, a frappée mr X sur le bras car elle n'était pas contente qu'il se lève,\
Se plaind toujours de céphalées, suite à son AEG, as mall a ses genoux, a vomi, particulièrement en colère contre les autres résidents,\
 Mr Y J'lemmerde et le personnel. Au cours du repas elle a jeté sa canne dans le petit salon, a un bleu sur le haut du front,\
risque de tomber, a uriné+++, lit souillé d'urine protection saturée, Bien enrhumée +++, souffre d'un cor au pied et demande un panst,\
agité +++ ce soir au change à crié, opposant++, a perdu son alliance, lance les objets. Son comportement devient difficile à gerer,\
de nouveau nauséeuse, se met à crier, tres endormi, petite plaie, les IDE sur place ont trouvé Y par terre, Oeil droit irrité (rouge),\
selles dures +++, a le nez qui coule et la voix cassé, pas sorti de ca chambre de la journeé, trés rouge au niveau des parties,\
tres fatiguer se matin, elle c'est endormi dvnt son ptit dejeuner, avait des vertiges ne se sentait pas bien est resté au lit,\
petite perte de sang vaginale, pas dormis, lit trempé +++ à deux reprises, a peu dormi, Tres angoisee, diahrée importante,\
se plaint de ne pouvoir aller à la selle, dit ne pas aimé celui qu on lui a mit,installé sur un fauteuil sans coussin anti escarre,\
bronchite, petit vomissement, refuse de se coucher, pas contente s'est mise à me parler méchament, parole bléssante,\
nombreues érosoins et rougeurs aux bras, dos, fesses et jambes. N'a rien mangé ce soir, Elle a crié toute la nuit ,perturbée+++,\
a rien dormi cette nuit enleve sa pull-ups est urine partout, très récalcitrante, dit qu'il est inadmissible d'être changer en pleine nuit!\
désorienté, Dermabrasion suite à tic d'arrachage pulpe annulaire gauche, pulpe majeur gauche, pas satisfaite de la réponse,\
 lésion vulvaire, lit mouillé, surveiller+++,  eventuelle contention au fauteuil, a arraché sa sonde urinaire, tjs perturbé,\
 suit le personnel  même dans les chambres des résidents aux moments des toilettes, Est très spastique, a peu dormi cette nuit , réveillée à plusieurs reprises,\
dorsalgies, douleurs nocturnes, IM connue, scoliose et tassements vertébraux. Ostéoporose connue non traitée, se plaint ce matin d 'avoir une douloureux a son bras droit,\
retrouvé en suspention dans le sucuridraps a coté du matela, très agitée, déhambule +++, a souillé son lit, Plaie(s), douloureux +++,\
a du mal a marcher, a failli tomber, a une posture penchée vers l'avant, durese 500 cc, rougeur, hématomes, Dit etre nauséeuse,\
39.2°C, Selles urines +++ à arracher sa protection, DCD, plaie fibrineuse, refus du petit dejeuner, dort +++, peu d'amelioration de l'état général,\
mycose, selles liquides, très douloureuse, cris, deshydratée , déambule dans les couloirs,cherche ses parents , aggresive , ne veut pas boire,\
 nous prend pour des folles, se plaint de céphalés, dire ne pas se sentir bien, Se plaint beaucoup en disant aie aie aie,\
odeurs d urine+++,  n'a pas manger ce midi, dort  bien  rarement, nous appel  la nuit,  refuse  de boire, refus, toujours en insuffisance cardiaque,\
a vomi, Hémoglobine très basse, asthénie, J3 sans selle, non collaboration, bouscule le chariot d'hydratation, insultes, menaces de nous frapper, s'énerve encore, \
 trés peu dormie, très grosse fatigue, malaise, encombrement bronchique,  toussant, plainte, pace maker, fatiguée, entorse,\
ne faut plus insiter, peu dormi, essayons de la calmer, très encombré, difficultés à avaler, oèdemes, rouge et chaud dlr,\
pas de selles, trop fatiguée, trés fatiguée, peniblement, trés ballonnée ce jour, couché dans son fauteuil, ne dormait pas,\
Comportement inhabitue, yeux fermer, évite de les enlever,  baisser la nuit, refusé, mal dormi, crie beaucoup, urines mal ododrante, selles liquides +++,\
fatigué, perturbé, il crit, deambulait, 4 ème jour sans selles, talonssacrum entammé, pastilles de nécrose aux deux talons, douloureux,\
peur,  se tape la tête dans les barrières de son lit, n'a pas uriné, pas eu de selle, pour uriné ça le brulé comme du feu,\
gaz +++, se plaint d'avoir pleins de médicaments, tres desagréable avec moi, état de fatigue, micose, petit hématome a l arcade coté droit,\
a peu dormi, deux jours sans selles, opposante tape, crache, hyperthermie, thymie très basse, dit se sentir seule et isolée,\
nauséeuse, na pas eu sa douche se jour personnel restrint, Certificat de décès, très endormie ce matin, n'arrive pas à déjeuner,\
tres agressive, se serait frotté la main, ventre gonflé"


In [11]:
def remove_stop_words(reviews):
    reviews = re.sub(r"[-'&@¨^.;:!*=\%?,#<>\"()\[\]]",' ', reviews)
    reviews = reviews.replace('ttt', 'traitement') 
    reviews = re.sub(r"([+]+)", ' plus ', reviews)
    #reviews = re.sub(r"([+])", ' ', reviews)
    '''
    for word in reviews.split() :
        if word in stop_words or len(word) < 2:
            word = r'\b' + word + r'\b'
            reviews = re.sub(word, '', reviews)
    '''
    reviews = re.sub(' +', ' ', reviews)
    reviews = re.sub('^\s+', '', reviews)
    return reviews  

In [113]:
# nagative : 0
# positive : 1
neg_train = [(remove_stop_words(word) , 0) for word in nagative.split(',')]
pos_train = [(remove_stop_words(word) , 1)for word in positive.split(',')]
init_X = neg_train + pos_train

numpy.random.shuffle(init_X)
X = [y[0] for y in init_X]
Y = [y[1] for y in init_X]
X[:10],Y[:10]

(['Diarrhées persistantes malgré fin du traitement antibiotique',
  'lit souillé d urine protection saturée',
  'trés rouge au niveau des parties',
  'cherche ses parents ',
  'Selles urines plus à arracher sa protection',
  'hématomes',
  'retire son bracelet régulièrement',
  'dire ne pas se sentir bien',
  'peniblement',
  'encore au plus mal avec crise de larmes et en me demandant de la tuer'],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [104]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer
#tokenizer to remove unwanted elements from out data like symbols and numbers
token = RegexpTokenizer(r'[a-zA-Z0-9]+')
cv = CountVectorizer(lowercase=True,stop_words=stop_words,ngram_range = (1,1),tokenizer = token.tokenize)
text_counts = cv.fit_transform(X)

In [105]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(text_counts, Y, test_size=0.3, random_state=1)

#y_train

# RandomForest

In [106]:
from sklearn.ensemble import RandomForestClassifier 

classifier=RandomForestClassifier(n_estimators=10)                  
classifier.fit(X_train.toarray(), y_train)                            
prediction = classifier.predict(X_test.toarray()) 
print(metrics.accuracy_score(y_test, prediction))

0.7870370370370371


# NaiveBayes

In [109]:
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB().fit(X_train, y_train)
predicted= clf.predict(X_test)
print("MultinomialNB Accuracy :",metrics.accuracy_score(y_test, predicted))

MultinomialNB Accuracy : 0.7962962962962963


In [110]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer()
text_tf= tf.fit_transform(X)

In [111]:
X_train, X_test, y_train, y_test = train_test_split(text_tf, Y, test_size=0.3, random_state=1)
#y_train

In [112]:
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

clf = MultinomialNB().fit(X_train, y_train)
predicted= clf.predict(X_test)
print("MultinomialNB Accuracy:",metrics.accuracy_score(y_test, predicted))

MultinomialNB Accuracy: 0.7870370370370371


# NaiveBayesClassifier with textblob

In [14]:
classifier = NaiveBayesClassifier(x_train)
print(classifier.accuracy(x_test))

0.7962962962962963


In [15]:
classifier.classify(('PERTE VAGINALE SANGUINOLANTE'.lower()))

'nagative'